<h1> Load in a data frame which has all the data

In [1]:
%%pyspark
df = spark.read.load('abfss://raw@synapseinaday.dfs.core.windows.net/TaxiNP/', format='parquet')
display(df.limit(10))

StatementMeta(SampleSpark, 8, 1, Finished, Available)

SynapseWidget(Synapse.DataFrame, cd270fa4-00bc-447c-b3fd-ef8871b78b9e)

<h1> Partition a dataframe

We use the partitionby function to partition by year

In [ ]:
WritePartitionedData = (
    df
    .write
    .partitionBy('PULocationID')
    .format("parquet")
    .save("abfss://precon@synapseinaday.dfs.core.windows.net/Demo6/PartionedByPULocationID/")
)


StatementMeta(, , , Cancelled, )

<h1>Creating a Partitioned Sorted Dataset

We use the coalesce to limit the amount of files and the partitionby to create the folder structure

In [6]:
df.createOrReplaceTempView('_temp_OrderedTaxi')
tbr = spark.sql('Select *, YEAR(dropoffdate) as TheYear from _temp_OrderedTaxi')
(tbr
.repartition("TheYear")
.sortWithinPartitions('PULocationID')
.write
.partitionBy('TheYear')
.format('parquet')
.mode("overwrite")
.save("abfss://precon@synapseinaday.dfs.core.windows.net/Demo6/SegmentE4/")
)

StatementMeta(SampleSpark, 8, 6, Submitted, Running)

<h1> Check for Spark SQL and see difference in file size and runtime

In [ ]:
Sorteddf = df.orderBy('PULocationID')

WriteSortedData = Sorteddf.write.format("parquet").save("abfss://precon@synapseinaday.dfs.core.windows.net/Demo6/OrderedByLocationID/")

In [3]:
df.createOrReplaceTempView('_temp_OrderedTaxi')
tbr = spark.sql('Select *, YEAR(dropoffdate) as TheYear from _temp_OrderedTaxi order by PULocationID,PickupDate')
tbr.write.partitionBy('PULocationID').format('parquet').save("abfss://precon@synapseinaday.dfs.core.windows.net/Demo6/OrderedAndSortedSparkSQL/")

StatementMeta(SampleSpark, 1, 3, Finished, Available)